In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
import ast
warnings.filterwarnings('ignore')
from langdetect import detect
import re
from scipy import stats

Matplotlib is building the font cache; this may take a moment.


# FUNCIÓN PARA LIMPIAR DATASETS, AÑADIR COLUMNAS DE NOMBRES DE DISTRITOS Y EXPORTARLOS.

Process_and_clean_dataset_yellow procesa los datasets de yellow trip. 

Process_and_clean_dataset_green y Process_and_clean_dataset_gr procesan los datasets de green trip

In [181]:
import pandas as pd

def process_and_clean_dataset_yellow(file_path, output_path):
    # Cargar el archivo CSV
    df = pd.read_parquet(file_path)

    # Definir el diccionario de mapeo
    pulocationid_to_borough = {
        161: 'Queens',43: 'Bronx',48: 'Manhattan',138: 'Queens',107: 'Brooklyn',239: 'Manhattan',142: 'Queens',164: 'Brooklyn',
        141: 'Queens',234: 'Manhattan',79: 'Brooklyn',33: 'Brooklyn',90: 'Brooklyn',113: 'Queens',237: 'Manhattan',143: 'Queens',
        137: 'Queens',132: 'Queens',163: 'Brooklyn',68: 'Brooklyn',263: 'Manhattan',170: 'Brooklyn',233: 'Manhattan',236: 'Queens',
        144: 'Queens',229: 'Manhattan',249: 'Manhattan',162: 'Queens',224: 'Manhattan',151: 'Queens',211: 'Manhattan',148: 'Queens',
        238: 'Manhattan',231: 'Manhattan',13: 'Manhattan',262: 'Manhattan',75: 'Brooklyn',264: 'Manhattan',125: 'Queens',186: 'Queens',
        158: 'Queens',246: 'Manhattan',50: 'Brooklyn',4: 'Manhattan',87: 'Brooklyn',114: 'Queens',24: 'Manhattan',166: 'Brooklyn',
        41: 'Manhattan',25: 'Manhattan',194: 'Manhattan',74: 'Brooklyn',140: 'Queens',146: 'Queens',65: 'Brooklyn',181: 'Brooklyn',
        255: 'Manhattan',256: 'Manhattan',88: 'Brooklyn',232: 'Manhattan',193: 'Manhattan',226: 'Manhattan',97: 'Brooklyn',
        197: 'Manhattan',37: 'Manhattan',189: 'Queens',261: 'Manhattan',225: 'Manhattan',244: 'Manhattan',45: 'Manhattan',40: 'Manhattan',
        116: 'Queens',80: 'Brooklyn',209: 'Manhattan',243: 'Manhattan',60: 'Brooklyn',265: 'Manhattan',100: 'Brooklyn',70: 'Brooklyn',
        188: 'Queens',52: 'Brooklyn',7: 'Manhattan',159: 'Queens',66: 'Brooklyn',42: 'Manhattan',174: 'Queens',215: 'Manhattan',
        252: 'Manhattan',136: 'Queens',223: 'Manhattan',93: 'Brooklyn',12: 'Manhattan',39: 'Manhattan',122: 'Queens',195: 'Manhattan',
        61: 'Brooklyn',112: 'Queens',230: 'Manhattan',82: 'Brooklyn',106: 'Brooklyn',228: 'Manhattan',179: 'Queens',260: 'Manhattan',
        152: 'Queens',121: 'Queens',49: 'Brooklyn',198: 'Manhattan',83: 'Brooklyn',17: 'Manhattan',168: 'Queens',56: 'Brooklyn',
        119: 'Queens',145: 'Queens',129: 'Queens',36: 'Manhattan', 1: 'Manhattan',95: 'Brooklyn',34: 'Manhattan',160: 'Queens',
        196: 'Manhattan',10: 'Manhattan',102: 'Brooklyn',247: 'Manhattan',86: 'Brooklyn',28: 'Manhattan',54: 'Brooklyn',77: 'Brooklyn',
        240: 'Manhattan',32: 'Manhattan',218: 'Manhattan',219: 'Manhattan',133: 'Queens',89: 'Brooklyn',156: 'Queens',177: 'Queens',
        85: 'Brooklyn',173: 'Queens',69: 'Brooklyn',14: 'Manhattan',51: 'Brooklyn',202: 'Manhattan',165: 'Queens',254: 'Manhattan',
        235: 'Manhattan',127: 'Queens',131: 'Queens',191: 'Queens',157: 'Queens',130: 'Queens',182: 'Queens',241: 'Manhattan',
        11: 'Manhattan',62: 'Brooklyn',258: 'Manhattan',169: 'Queens',123: 'Queens',47: 'Manhattan',6: 'Manhattan',22: 'Manhattan',
        135: 'Queens',216: 'Manhattan',155: 'Queens',63: 'Brooklyn',91: 'Brooklyn',94: 'Brooklyn',108: 'Queens',92: 'Brooklyn',
        44: 'Manhattan',259: 'Manhattan',124: 'Queens',78: 'Brooklyn',171: 'Queens',149: 'Queens',55: 'Brooklyn',180: 'Brooklyn',
        35: 'Manhattan',81: 'Brooklyn',257: 'Manhattan',139: 'Queens',72: 'Brooklyn',71: 'Brooklyn',19: 'Manhattan',73: 'Brooklyn',
        200: 'Manhattan',222: 'Manhattan',208: 'Manhattan',76: 'Brooklyn',210: 'Manhattan',15: 'Manhattan',147: 'Queens',167: 'Queens',
        126: 'Queens',213: 'Manhattan',5: 'Manhattan',96: 'Brooklyn',248: 'Manhattan',205: 'Manhattan',21: 'Manhattan',250: 'Manhattan',
        153: 'Queens',18: 'Manhattan',134: 'Queens',242: 'Manhattan',192: 'Manhattan',98: 'Brooklyn',8: 'Manhattan',31: 'Manhattan',
        26: 'Manhattan',67: 'Brooklyn',203: 'Manhattan',201: 'Manhattan',30: 'Manhattan',220: 'Manhattan',227: 'Manhattan',53: 'Brooklyn',
        20: 'Manhattan',38: 'Manhattan',185: 'Queens',251: 'Manhattan',212: 'Manhattan',183: 'Queens',207: 'Manhattan',9: 'Manhattan',
        175: 'Queens',16: 'Manhattan',23: 'Manhattan',3: 'Manhattan',117: 'Queens',64: 'Brooklyn',217: 'Manhattan',253: 'Manhattan',
        190: 'Queens',111: 'Queens',115: 'Queens',29: 'Manhattan',101: 'Brooklyn',154: 'Queens',178: 'Queens',150: 'Queens',57: 'Brooklyn',
        120: 'Queens',199: 'Manhattan',214: 'Manhattan',128: 'Queens',187: 'Queens',221: 'Manhattan',118: 'Queens',206: 'Manhattan',
        109: 'Queens',184: 'Queens',172: 'Queens',99: 'Brooklyn',245: 'Manhattan',46: 'Manhattan',2: 'Manhattan',58: 'Brooklyn'
        }


    # Crear nuevas columnas mapeando 'PULocationID' y 'DOLocationID' a 'Boroughs'
    df['PULBoroughs'] = df['PULocationID'].map(pulocationid_to_borough)
    df['DOLBoroughs'] = df['DOLocationID'].map(pulocationid_to_borough)
    print(df[['PULocationID', 'PULBoroughs', 'DOLocationID', 'DOLBoroughs']].head())

    # Eliminar columnas con más del 50% de valores nulos
    df.dropna(thresh=0.5*len(df), axis=1, inplace=True)

    # Eliminar duplicados
    df.drop_duplicates(subset=['tpep_pickup_datetime'], inplace=True)

    # Eliminar columnas que no serán empleadas
    df = df[['VendorID', 'tpep_pickup_datetime','tpep_dropoff_datetime','PULocationID','PULBoroughs', 'DOLocationID', 'DOLBoroughs', 'trip_distance', 'fare_amount', 'total_amount','tolls_amount', 'payment_type']]

    # Guardar el dataframe limpio en un nuevo archivo CSV
    df.to_csv(output_path, index=False)

    return df

Implementación

In [ ]:
file_path = 'yellow_tripdata_2021-12.parquet'
output_path = 'yellowtripdata202112.csv'
df_processed_and_cleaned = process_and_clean_dataset_yellow(file_path, output_path)
print(df_processed_and_cleaned.head())


   PULocationID PULBoroughs  DOLocationID DOLBoroughs
0           138      Queens           141      Queens
1           238   Manhattan            42   Manhattan
2           239   Manhattan            74    Brooklyn
3           148      Queens            87    Brooklyn
4           231   Manhattan           246   Manhattan
   VendorID tpep_pickup_datetime tpep_dropoff_datetime  PULocationID  \
0         1  2021-12-01 00:19:51   2021-12-01 00:37:01           138   
1         2  2021-12-01 00:29:07   2021-12-01 00:45:13           238   
2         1  2021-12-01 00:12:40   2021-12-01 00:27:17           239   
3         1  2021-12-01 00:10:18   2021-12-01 00:19:20           148   
4         1  2021-12-01 00:25:12   2021-12-01 00:39:07           231   

  PULBoroughs  DOLocationID DOLBoroughs  trip_distance  fare_amount  \
0      Queens           141      Queens           9.30         26.5   
1   Manhattan            42   Manhattan           2.76         13.0   
2   Manhattan            74   

In [147]:
import pandas as pd

def process_and_clean_dataset_gr(file_path, output_path):
    # Cargar el archivo CSV
    df = pd.read_parquet(file_path)

    # Definir el diccionario de mapeo
    pulocationid_to_borough = {
        161: 'Queens',43: 'Bronx',48: 'Manhattan',138: 'Queens',107: 'Brooklyn',239: 'Manhattan',142: 'Queens',164: 'Brooklyn',
        141: 'Queens',234: 'Manhattan',79: 'Brooklyn',33: 'Brooklyn',90: 'Brooklyn',113: 'Queens',237: 'Manhattan',143: 'Queens',
        137: 'Queens',132: 'Queens',163: 'Brooklyn',68: 'Brooklyn',263: 'Manhattan',170: 'Brooklyn',233: 'Manhattan',236: 'Queens',
        144: 'Queens',229: 'Manhattan',249: 'Manhattan',162: 'Queens',224: 'Manhattan',151: 'Queens',211: 'Manhattan',148: 'Queens',
        238: 'Manhattan',231: 'Manhattan',13: 'Manhattan',262: 'Manhattan',75: 'Brooklyn',264: 'Manhattan',125: 'Queens',186: 'Queens',
        158: 'Queens',246: 'Manhattan',50: 'Brooklyn',4: 'Manhattan',87: 'Brooklyn',114: 'Queens',24: 'Manhattan',166: 'Brooklyn',
        41: 'Manhattan',25: 'Manhattan',194: 'Manhattan',74: 'Brooklyn',140: 'Queens',146: 'Queens',65: 'Brooklyn',181: 'Brooklyn',
        255: 'Manhattan',256: 'Manhattan',88: 'Brooklyn',232: 'Manhattan',193: 'Manhattan',226: 'Manhattan',97: 'Brooklyn',
        197: 'Manhattan',37: 'Manhattan',189: 'Queens',261: 'Manhattan',225: 'Manhattan',244: 'Manhattan',45: 'Manhattan',40: 'Manhattan',
        116: 'Queens',80: 'Brooklyn',209: 'Manhattan',243: 'Manhattan',60: 'Brooklyn',265: 'Manhattan',100: 'Brooklyn',70: 'Brooklyn',
        188: 'Queens',52: 'Brooklyn',7: 'Manhattan',159: 'Queens',66: 'Brooklyn',42: 'Manhattan',174: 'Queens',215: 'Manhattan',
        252: 'Manhattan',136: 'Queens',223: 'Manhattan',93: 'Brooklyn',12: 'Manhattan',39: 'Manhattan',122: 'Queens',195: 'Manhattan',
        61: 'Brooklyn',112: 'Queens',230: 'Manhattan',82: 'Brooklyn',106: 'Brooklyn',228: 'Manhattan',179: 'Queens',260: 'Manhattan',
        152: 'Queens',121: 'Queens',49: 'Brooklyn',198: 'Manhattan',83: 'Brooklyn',17: 'Manhattan',168: 'Queens',56: 'Brooklyn',
        119: 'Queens',145: 'Queens',129: 'Queens',36: 'Manhattan', 1: 'Manhattan',95: 'Brooklyn',34: 'Manhattan',160: 'Queens',
        196: 'Manhattan',10: 'Manhattan',102: 'Brooklyn',247: 'Manhattan',86: 'Brooklyn',28: 'Manhattan',54: 'Brooklyn',77: 'Brooklyn',
        240: 'Manhattan',32: 'Manhattan',218: 'Manhattan',219: 'Manhattan',133: 'Queens',89: 'Brooklyn',156: 'Queens',177: 'Queens',
        85: 'Brooklyn',173: 'Queens',69: 'Brooklyn',14: 'Manhattan',51: 'Brooklyn',202: 'Manhattan',165: 'Queens',254: 'Manhattan',
        235: 'Manhattan',127: 'Queens',131: 'Queens',191: 'Queens',157: 'Queens',130: 'Queens',182: 'Queens',241: 'Manhattan',
        11: 'Manhattan',62: 'Brooklyn',258: 'Manhattan',169: 'Queens',123: 'Queens',47: 'Manhattan',6: 'Manhattan',22: 'Manhattan',
        135: 'Queens',216: 'Manhattan',155: 'Queens',63: 'Brooklyn',91: 'Brooklyn',94: 'Brooklyn',108: 'Queens',92: 'Brooklyn',
        44: 'Manhattan',259: 'Manhattan',124: 'Queens',78: 'Brooklyn',171: 'Queens',149: 'Queens',55: 'Brooklyn',180: 'Brooklyn',
        35: 'Manhattan',81: 'Brooklyn',257: 'Manhattan',139: 'Queens',72: 'Brooklyn',71: 'Brooklyn',19: 'Manhattan',73: 'Brooklyn',
        200: 'Manhattan',222: 'Manhattan',208: 'Manhattan',76: 'Brooklyn',210: 'Manhattan',15: 'Manhattan',147: 'Queens',167: 'Queens',
        126: 'Queens',213: 'Manhattan',5: 'Manhattan',96: 'Brooklyn',248: 'Manhattan',205: 'Manhattan',21: 'Manhattan',250: 'Manhattan',
        153: 'Queens',18: 'Manhattan',134: 'Queens',242: 'Manhattan',192: 'Manhattan',98: 'Brooklyn',8: 'Manhattan',31: 'Manhattan',
        26: 'Manhattan',67: 'Brooklyn',203: 'Manhattan',201: 'Manhattan',30: 'Manhattan',220: 'Manhattan',227: 'Manhattan',53: 'Brooklyn',
        20: 'Manhattan',38: 'Manhattan',185: 'Queens',251: 'Manhattan',212: 'Manhattan',183: 'Queens',207: 'Manhattan',9: 'Manhattan',
        175: 'Queens',16: 'Manhattan',23: 'Manhattan',3: 'Manhattan',117: 'Queens',64: 'Brooklyn',217: 'Manhattan',253: 'Manhattan',
        190: 'Queens',111: 'Queens',115: 'Queens',29: 'Manhattan',101: 'Brooklyn',154: 'Queens',178: 'Queens',150: 'Queens',57: 'Brooklyn',
        120: 'Queens',199: 'Manhattan',214: 'Manhattan',128: 'Queens',187: 'Queens',221: 'Manhattan',118: 'Queens',206: 'Manhattan',
        109: 'Queens',184: 'Queens',172: 'Queens',99: 'Brooklyn',245: 'Manhattan',46: 'Manhattan',2: 'Manhattan',58: 'Brooklyn'
        }


    # Crear nuevas columnas mapeando 'PULocationID' y 'DOLocationID' a 'Boroughs'
    df['PULBoroughs'] = df['PULocationID'].map(pulocationid_to_borough)
    df['DOLBoroughs'] = df['DOLocationID'].map(pulocationid_to_borough)
    print(df[['PULocationID', 'PULBoroughs', 'DOLocationID', 'DOLBoroughs']].head())

    # Eliminar columnas con más del 50% de valores nulos
    df.dropna(thresh=0.5*len(df), axis=1, inplace=True)

    # Eliminar duplicados
    df.drop_duplicates(subset=['lpep_pickup_datetime'], inplace=True)

    # Eliminar columnas que no serán empleadas
    df = df[['VendorID', 'lpep_pickup_datetime','lpep_dropoff_datetime','PULocationID','PULBoroughs', 'DOLocationID', 'DOLBoroughs', 'trip_distance', 'fare_amount', 'total_amount','tolls_amount', 'payment_type']]

    # Guardar el dataframe limpio en un nuevo archivo CSV
    df.to_csv(output_path, index=False)

    return df

Implementación

In [192]:
file_path = 'green_tripdata_2021-02.parquet'
output_path = 'greentrip202102.csv'
df_processed_and_cleaned = process_and_clean_dataset_gr(file_path, output_path)
print(df_processed_and_cleaned.head())

   PULocationID PULBoroughs  DOLocationID DOLBoroughs
0           130      Queens           205   Manhattan
1           152      Queens           244   Manhattan
2           152      Queens            48   Manhattan
3           152      Queens           241   Manhattan
4            75    Brooklyn            42   Manhattan
   VendorID lpep_pickup_datetime lpep_dropoff_datetime  PULocationID  \
0         2  2021-02-01 00:34:03   2021-02-01 00:51:58           130   
1         2  2021-02-01 00:04:00   2021-02-01 00:10:30           152   
2         2  2021-02-01 00:18:51   2021-02-01 00:34:06           152   
3         2  2021-02-01 00:53:27   2021-02-01 01:11:41           152   
4         2  2021-02-01 00:57:46   2021-02-01 01:06:44            75   

  PULBoroughs  DOLocationID DOLBoroughs  trip_distance  fare_amount  \
0      Queens           205   Manhattan           3.66         14.0   
1      Queens           244   Manhattan           1.10          6.5   
2      Queens            48   

In [180]:
import pandas as pd

def process_and_clean_dataset_green(file_path, output_path):
    # Cargar el archivo
    df = pd.read_parquet(file_path)

    # Definir el diccionario de mapeo
    pulocationid_to_borough = {
        161: 'Queens',43: 'Bronx',48: 'Manhattan',138: 'Queens',107: 'Brooklyn',239: 'Manhattan',142: 'Queens',164: 'Brooklyn',
        141: 'Queens',234: 'Manhattan',79: 'Brooklyn',33: 'Brooklyn',90: 'Brooklyn',113: 'Queens',237: 'Manhattan',143: 'Queens',
        137: 'Queens',132: 'Queens',163: 'Brooklyn',68: 'Brooklyn',263: 'Manhattan',170: 'Brooklyn',233: 'Manhattan',236: 'Queens',
        144: 'Queens',229: 'Manhattan',249: 'Manhattan',162: 'Queens',224: 'Manhattan',151: 'Queens',211: 'Manhattan',148: 'Queens',
        238: 'Manhattan',231: 'Manhattan',13: 'Manhattan',262: 'Manhattan',75: 'Brooklyn',264: 'Manhattan',125: 'Queens',186: 'Queens',
        158: 'Queens',246: 'Manhattan',50: 'Brooklyn',4: 'Manhattan',87: 'Brooklyn',114: 'Queens',24: 'Manhattan',166: 'Brooklyn',
        41: 'Manhattan',25: 'Manhattan',194: 'Manhattan',74: 'Brooklyn',140: 'Queens',146: 'Queens',65: 'Brooklyn',181: 'Brooklyn',
        255: 'Manhattan',256: 'Manhattan',88: 'Brooklyn',232: 'Manhattan',193: 'Manhattan',226: 'Manhattan',97: 'Brooklyn',
        197: 'Manhattan',37: 'Manhattan',189: 'Queens',261: 'Manhattan',225: 'Manhattan',244: 'Manhattan',45: 'Manhattan',40: 'Manhattan',
        116: 'Queens',80: 'Brooklyn',209: 'Manhattan',243: 'Manhattan',60: 'Brooklyn',265: 'Manhattan',100: 'Brooklyn',70: 'Brooklyn',
        188: 'Queens',52: 'Brooklyn',7: 'Manhattan',159: 'Queens',66: 'Brooklyn',42: 'Manhattan',174: 'Queens',215: 'Manhattan',
        252: 'Manhattan',136: 'Queens',223: 'Manhattan',93: 'Brooklyn',12: 'Manhattan',39: 'Manhattan',122: 'Queens',195: 'Manhattan',
        61: 'Brooklyn',112: 'Queens',230: 'Manhattan',82: 'Brooklyn',106: 'Brooklyn',228: 'Manhattan',179: 'Queens',260: 'Manhattan',
        152: 'Queens',121: 'Queens',49: 'Brooklyn',198: 'Manhattan',83: 'Brooklyn',17: 'Manhattan',168: 'Queens',56: 'Brooklyn',
        119: 'Queens',145: 'Queens',129: 'Queens',36: 'Manhattan', 1: 'Manhattan',95: 'Brooklyn',34: 'Manhattan',160: 'Queens',
        196: 'Manhattan',10: 'Manhattan',102: 'Brooklyn',247: 'Manhattan',86: 'Brooklyn',28: 'Manhattan',54: 'Brooklyn',77: 'Brooklyn',
        240: 'Manhattan',32: 'Manhattan',218: 'Manhattan',219: 'Manhattan',133: 'Queens',89: 'Brooklyn',156: 'Queens',177: 'Queens',
        85: 'Brooklyn',173: 'Queens',69: 'Brooklyn',14: 'Manhattan',51: 'Brooklyn',202: 'Manhattan',165: 'Queens',254: 'Manhattan',
        235: 'Manhattan',127: 'Queens',131: 'Queens',191: 'Queens',157: 'Queens',130: 'Queens',182: 'Queens',241: 'Manhattan',
        11: 'Manhattan',62: 'Brooklyn',258: 'Manhattan',169: 'Queens',123: 'Queens',47: 'Manhattan',6: 'Manhattan',22: 'Manhattan',
        135: 'Queens',216: 'Manhattan',155: 'Queens',63: 'Brooklyn',91: 'Brooklyn',94: 'Brooklyn',108: 'Queens',92: 'Brooklyn',
        44: 'Manhattan',259: 'Manhattan',124: 'Queens',78: 'Brooklyn',171: 'Queens',149: 'Queens',55: 'Brooklyn',180: 'Brooklyn',
        35: 'Manhattan',81: 'Brooklyn',257: 'Manhattan',139: 'Queens',72: 'Brooklyn',71: 'Brooklyn',19: 'Manhattan',73: 'Brooklyn',
        200: 'Manhattan',222: 'Manhattan',208: 'Manhattan',76: 'Brooklyn',210: 'Manhattan',15: 'Manhattan',147: 'Queens',167: 'Queens',
        126: 'Queens',213: 'Manhattan',5: 'Manhattan',96: 'Brooklyn',248: 'Manhattan',205: 'Manhattan',21: 'Manhattan',250: 'Manhattan',
        153: 'Queens',18: 'Manhattan',134: 'Queens',242: 'Manhattan',192: 'Manhattan',98: 'Brooklyn',8: 'Manhattan',31: 'Manhattan',
        26: 'Manhattan',67: 'Brooklyn',203: 'Manhattan',201: 'Manhattan',30: 'Manhattan',220: 'Manhattan',227: 'Manhattan',53: 'Brooklyn',
        20: 'Manhattan',38: 'Manhattan',185: 'Queens',251: 'Manhattan',212: 'Manhattan',183: 'Queens',207: 'Manhattan',9: 'Manhattan',
        175: 'Queens',16: 'Manhattan',23: 'Manhattan',3: 'Manhattan',117: 'Queens',64: 'Brooklyn',217: 'Manhattan',253: 'Manhattan',
        190: 'Queens',111: 'Queens',115: 'Queens',29: 'Manhattan',101: 'Brooklyn',154: 'Queens',178: 'Queens',150: 'Queens',57: 'Brooklyn',
        120: 'Queens',199: 'Manhattan',214: 'Manhattan',128: 'Queens',187: 'Queens',221: 'Manhattan',118: 'Queens',206: 'Manhattan',
        109: 'Queens',184: 'Queens',172: 'Queens',99: 'Brooklyn',245: 'Manhattan',46: 'Manhattan',2: 'Manhattan',58: 'Brooklyn'
    }

    # Crear nuevas columnas mapeando 'PULocationID' y 'DOLocationID' a 'Boroughs'
    df['PULBoroughs'] = df['PULocationID'].map(pulocationid_to_borough)
    df['DOLBoroughs'] = df['DOLocationID'].map(pulocationid_to_borough)
    print(df[['PULocationID', 'PULBoroughs', 'DOLocationID', 'DOLBoroughs']].head())

    # Eliminar columnas con más del 50% de valores nulos
    df.dropna(thresh=0.5*len(df), axis=1, inplace=True)

    # Eliminar duplicados
    df.drop_duplicates(subset=['lpep_pickup_datetime'], inplace=True)

    # Eliminar columnas que no serán empleadas
    df = df[['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime','PULocationID', 'PULBoroughs', 'DOLocationID', 'DOLBoroughs', 'trip_distance', 'fare_amount','total_amount','tolls_amount']]

    # Guardar el dataframe limpio en un nuevo archivo CSV
    df.to_csv(output_path, index=False)

    return df

Implementación:

In [ ]:
file_path = 'green_tripdata_2021-12.parquet'
output_path = 'greentripdata202112.csv'
df_processed_and_cleaned = process_and_clean_dataset_green(file_path, output_path)
print(df_processed_and_cleaned.head())

   PULocationID PULBoroughs  DOLocationID DOLBoroughs
0            82    Brooklyn            82    Brooklyn
1           236      Queens           151      Queens
2           166    Brooklyn            75    Brooklyn
3            74    Brooklyn            75    Brooklyn
4            82    Brooklyn           129      Queens
   VendorID lpep_pickup_datetime lpep_dropoff_datetime  PULocationID  \
0         2  2021-12-01 00:35:15   2021-12-01 00:42:24            82   
1         2  2021-12-01 00:15:54   2021-12-01 00:20:31           236   
2         2  2021-12-01 00:33:46   2021-12-01 00:43:29           166   
3         1  2021-12-01 00:29:51   2021-12-01 00:35:50            74   
4         2  2021-12-01 00:13:06   2021-12-01 00:17:11            82   

  PULBoroughs  DOLocationID DOLBoroughs  trip_distance  fare_amount  \
0    Brooklyn            82    Brooklyn           0.65          6.0   
1      Queens           151      Queens           1.62          6.5   
2    Brooklyn            75   

# FUNCIÓN PARA MUESTREO DEL DATASET

In [193]:
import pandas as pd

def calcular_tamano_muestra(N, Z=1.96, p=0.5, d=0.05):
    """Calcula el tamaño de la muestra."""
    return int((N * (Z**2) * p * (1 - p)) / ((d**2) * (N - 1) + (Z**2) * p * (1 - p)))

def procesar_muestra(archivo_entrada, archivo_salida):
    """Procesa una muestra del DataFrame y la guarda en un archivo CSV."""
    # Cargar el archivo CSV
    df = pd.read_csv(archivo_entrada)
    
    # Calcular el tamaño de la muestra
    N = len(df)
    tamano_muestra = calcular_tamano_muestra(N)
    print(f"Tamaño de la muestra: {tamano_muestra}")
    
    # Tomar una muestra aleatoria del DataFrame
    df_muestra = df.sample(n=tamano_muestra, random_state=1)
    
    # Guardar la muestra en un archivo CSV
    df_muestra.to_csv(archivo_salida, index=False)
    print(f"Muestra guardada en {archivo_salida}")

Implementación

In [238]:
archivo_entrada = 'greentrip202408.csv'
archivo_salida = 'greentripdata202408muestra.csv'
procesar_muestra(archivo_entrada, archivo_salida)

Tamaño de la muestra: 381
Muestra guardada en greentripdata202408muestra.csv


# FUNCIÓN PARA UNIR DATASETS

In [3]:
import pandas as pd

def unir_datasets(archivos, archivo_salida):
    """
    Une múltiples datasets que tienen las mismas columnas y guarda el resultado en un archivo CSV.

    Args:
        archivos (list): Lista de rutas a los archivos CSV a unir.
        archivo_salida (str): Ruta del archivo CSV de salida.
    """
    dataframes = [pd.read_csv(archivo) for archivo in archivos]
    df_unido = pd.concat(dataframes, ignore_index=True)
    df_unido.to_csv(archivo_salida, index=False)
    print(f"Datasets unidos y guardados en {archivo_salida}")

Implementación

In [4]:
archivos = ['greentripmuestranormalizada.csv','yellowtripmuestranormalizada.csv']
archivo_salida = 'taxismuestrasunido.csv'
unir_datasets(archivos, archivo_salida)


Datasets unidos y guardados en taxismuestrasunido.csv


In [5]:
m=pd.read_csv('taxismuestrasunido.csv')

In [7]:
m.shape

(33673, 14)

# FUNCIÓN PARA CAMBIAR NOMBRES DE COLUMNAS

In [139]:
import pandas as pd

def cambiar_nombres_columnas(df, nuevos_nombres):
    """
    Cambia los nombres de las columnas de un DataFrame según un diccionario dado.

    Args:
        df (pd.DataFrame): DataFrame cuyos nombres de columnas se van a cambiar.
        nuevos_nombres (dict): Diccionario con los nombres actuales como claves y los nuevos nombres como valores.

    Returns:
        pd.DataFrame: DataFrame con los nombres de columnas cambiados.
    """
    df = df.rename(columns=nuevos_nombres)
    return df



In [247]:
archivo.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,PULocationID,PULBoroughs,DOLocationID,DOLBoroughs,trip_distance,fare_amount,total_amount,tolls_amount,payment_type
0,2,2021-01-02 17:02:00,2021-01-02 17:11:00,97,Brooklyn,17,Manhattan,1.83,23.07,26.12,0.0,NaN
1,2,2021-01-09 10:20:26,2021-01-09 10:27:45,41,Manhattan,42,Manhattan,1.54,7.50,8.30,0.0,2.0
2,2,2021-01-29 18:28:00,2021-01-29 18:54:00,177,Queens,39,Manhattan,4.14,23.31,26.36,0.0,NaN
3,2,2021-01-06 16:21:47,2021-01-06 16:30:43,41,Manhattan,166,Brooklyn,1.67,8.50,11.30,0.0,1.0
4,1,2021-01-26 16:40:12,2021-01-26 16:54:51,41,Manhattan,168,Queens,2.90,12.00,17.90,0.0,1.0


Implementación

In [250]:
archivo = 'greentripmuestrasunido.csv'
df = pd.read_csv(archivo)

nuevos_nombres = {
    'VendorID': 'Proveedor TPEP',
    'lpep_pickup_datetime': 'Hora recogida',
    'lpep_dropoff_datetime':'Hora destino',
    'PULocationID ':'ID lugar recogida',
    'PULBoroughs':'Distrito recogida',
    'DOLocationID':'ID lugar llegada',
    'DOLBoroughs':'Distrito llegada',
    'trip_distance':'Distancia viaje',
    'fare_amount':'Costo tarifa',
    'total_amount':'Total',
    'tolls_amount':'Peajes',
    'payment_type':'Tipo pago',
}

df_renombrado = cambiar_nombres_columnas(df, nuevos_nombres)
print(df_renombrado.head())
df_renombrado.to_csv('greentripmuestranormalizada.csv')

   Proveedor TPEP        Hora recogida         Hora destino  PULocationID  \
0               2  2021-01-02 17:02:00  2021-01-02 17:11:00            97   
1               2  2021-01-09 10:20:26  2021-01-09 10:27:45            41   
2               2  2021-01-29 18:28:00  2021-01-29 18:54:00           177   
3               2  2021-01-06 16:21:47  2021-01-06 16:30:43            41   
4               1  2021-01-26 16:40:12  2021-01-26 16:54:51            41   

  Distrito recogida  ID lugar llegada Distrito llegada  Distancia viaje  \
0          Brooklyn                17        Manhattan             1.83   
1         Manhattan                42        Manhattan             1.54   
2            Queens                39        Manhattan             4.14   
3         Manhattan               166         Brooklyn             1.67   
4         Manhattan               168           Queens             2.90   

   Costo tarifa  Total  Peajes  Tipo pago  
0         23.07  26.12     0.0        NaN 

# FUNCIÓN PARA AUTOMATIZAR EL EDA

DEMASIADO PESADA, NO FUNCIONA

In [252]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO

def eda_automatica_html(ruta_archivo, ruta_salida_html):
    # Cargar el archivo
    df = pd.read_csv(ruta_archivo)

    # Crear un objeto StringIO para almacenar el HTML
    html_str = StringIO()

    # Resumen estadístico
    resumen = df.describe().to_html()

    # Crear el archivo HTML
    html_str.write('<html><head><title>EDA Automático</title></head><body>')
    html_str.write('<h1>Resumen Estadístico</h1>')
    html_str.write(resumen)

    # Histograma de cada columna numérica
    for columna in df.select_dtypes(include=['number']).columns:
        plt.figure(figsize=(10, 5))
        sns.histplot(df[columna], kde=True)
        plt.title(f'Histograma de {columna}')
        img_path = f'{columna}_histogram.png'
        plt.savefig(img_path)
        plt.close()
        html_str.write(f'<h2>Histograma de {columna}</h2>')
        html_str.write(f'<img src="{img_path}" alt="Histograma de {columna}">')

    # Gráficos de dispersión entre pares de variables
    sns.pairplot(df.select_dtypes(include=['number']), palette='crest', hue='DOLBoroughs')
    pairplot_path = 'pairplot.png'
    plt.savefig(pairplot_path)
    plt.close()
    html_str.write('<h2>Gráfico de dispersión entre pares de variables</h2>')
    html_str.write(f'<img src="{pairplot_path}" alt="Gráfico de dispersión entre pares de variables">')

    # Matriz de correlación
    matriz_corr = df.corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(matriz_corr, annot=True, cmap='crest')
    correlacion_path = 'matriz_correlacion.png'
    plt.title('Matriz de correlación')
    plt.savefig(correlacion_path)
    plt.close()
    html_str.write('<h2>Matriz de correlación</h2>')
    html_str.write(f'<img src="{correlacion_path}" alt="Matriz de correlación">')

    # Cerrar el archivo HTML
    html_str.write('</body></html>')

    # Guardar el HTML en un archivo
    with open(ruta_salida_html, 'w') as f:
        f.write(html_str.getvalue())

    print("EDA automática completada. Los resultados están guardados en el archivo HTML especificado.")


Implementación

# FUNCIÓN PARA CREAR COLUMNAS DE AÑO, MES Y DIA

In [21]:
import pandas as pd

def añadir_dia_semana_y_guardar(df, archivo_salida):
    # Convertir las columnas 'Hora recogida' y 'Hora destino' a datetime
    df['Hora recogida'] = pd.to_datetime(df['Hora recogida'])
    df['Hora destino'] = pd.to_datetime(df['Hora destino'])

    # Crear nuevas columnas para 'Hora recogida'
    df['Año recogida'] = df['Hora recogida'].dt.year
    df['Mes recogida'] = df['Hora recogida'].dt.month
    df['Día recogida'] = df['Hora recogida'].dt.day
    df['Día semana recogida'] = df['Hora recogida'].dt.day_name()
    # Guardar el DataFrame modificado en un nuevo archivo CSV
    df.to_csv(archivo_salida, index=False)
    print(f"Archivo guardado en {archivo_salida}")





Implementación

In [24]:
# Ejemplo de uso
archivo_entrada = 'yellowtripmuestranormalizada.csv'
archivo_salida = 'yellowtripmuestranormalizadamodificado.csv'

# Cargar el archivo CSV
yellow = pd.read_csv(archivo_entrada)

# Llamar a la función con el DataFrame y el nombre del archivo de salida
añadir_dia_semana_y_guardar(yellow, archivo_salida)

Archivo guardado en yellowtripmuestranormalizadamodificado.csv
